Connecting Drive to Colab
Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Setting up PySpark in Colab

In [116]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Installing Apache Spark 3.1.2 with Hadoop 3.2 from the link

In [120]:
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

To unzip that folder

In [121]:
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

Install findspark library

In [122]:
!pip install -q findspark

To set the environment path

In [123]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

To locate Spark in the system

In [124]:
import findspark
findspark.init()

To know the location where Spark is installed

In [125]:
findspark.find()

'/content/spark-3.1.2-bin-hadoop3.2'

To view the Spark UI

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2021-11-14 11:30:02--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.19M  16.2MB/s    in 0.8s    

2021-11-14 11:30:03 (16.2 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://add8-35-230-10-11.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,

In [126]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("League Analysis")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
print("A Sky Bet League Competition Analysis")

A Sky Bet League Competition Analysis


Read Sky_competition_clean.txt csv file into PySpark

In [127]:
match_data = spark.read.csv("/content/Sky_competition_clean.txt", header=True, inferSchema=True)

Understanding the Data

In [128]:
match_data.printSchema()

root
 |-- away_team_id: string (nullable = true)
 |-- away_team_name: string (nullable = true)
 |-- away_team_score: integer (nullable = true)
 |-- home_team_id: string (nullable = true)
 |-- home_team_name: string (nullable = true)
 |-- home_team_score: integer (nullable = true)
 |-- match_id: string (nullable = true)
 |-- match_period: string (nullable = true)
 |-- match_competition: string (nullable = true)
 |-- match_round: integer (nullable = true)
 |-- match_season: string (nullable = true)
 |-- match_seasonId: integer (nullable = true)
 |-- match_skyId: integer (nullable = true)
 |-- match_status: string (nullable = true)
 |-- match_type: string (nullable = true)
 |-- match_venue_id: integer (nullable = true)
 |-- match_venue_location: string (nullable = true)
 |-- match_venue_name: string (nullable = true)
 |-- _c18: string (nullable = true)



To see the physical plan 

In [129]:
match_data.explain()

== Physical Plan ==
FileScan csv [away_team_id#3574,away_team_name#3575,away_team_score#3576,home_team_id#3577,home_team_name#3578,home_team_score#3579,match_id#3580,match_period#3581,match_competition#3582,match_round#3583,match_season#3584,match_seasonId#3585,match_skyId#3586,match_status#3587,match_type#3588,match_venue_id#3589,match_venue_location#3590,match_venue_name#3591,_c18#3592] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/content/Sky_competition_clean.txt], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<away_team_id:string,away_team_name:string,away_team_score:int,home_team_id:string,home_tea...




Show top 5 rows

In [130]:
match_data.show(5)

+------------+---------------+---------------+------------+----------------+---------------+--------------------+------------+------------------+-----------+---------------+--------------+-----------+------------+----------+--------------+--------------------+--------------------+----+
|away_team_id| away_team_name|away_team_score|home_team_id|  home_team_name|home_team_score|            match_id|match_period| match_competition|match_round|   match_season|match_seasonId|match_skyId|match_status|match_type|match_venue_id|match_venue_location|    match_venue_name|_c18|
+------------+---------------+---------------+------------+----------------+---------------+--------------------+------------+------------------+-----------+---------------+--------------+-----------+------------+----------+--------------+--------------------+--------------------+----+
|         t25|  Middlesbrough|              0|         t57|         Watford|              1|7af1fb87-75e5-322...|    FullTime|SkyBetChampio

Drop the last column ('_c18')

In [206]:
#match_data_clean = match_data.drop('_c18').show(10)

Add new column (team_name)

In [137]:
match_data = match_data.withColumn("team_name", match_data.away_team_name)

In [138]:
match_data.show(5)

+------------+---------------+---------------+------------+----------------+---------------+--------------------+------------+------------------+-----------+---------------+--------------+-----------+------------+----------+--------------+--------------------+--------------------+----+---------------+
|away_team_id| away_team_name|away_team_score|home_team_id|  home_team_name|home_team_score|            match_id|match_period| match_competition|match_round|   match_season|match_seasonId|match_skyId|match_status|match_type|match_venue_id|match_venue_location|    match_venue_name|_c18|      team_name|
+------------+---------------+---------------+------------+----------------+---------------+--------------------+------------+------------------+-----------+---------------+--------------+-----------+------------+----------+--------------+--------------------+--------------------+----+---------------+
|         t25|  Middlesbrough|              0|         t57|         Watford|              1

Add new column (total_match_score)

In [141]:
match_data = match_data.withColumn("total_match_score", match_data.away_team_score + match_data.home_team_score)

In [142]:
match_data.show(5)

+------------+---------------+---------------+------------+----------------+---------------+--------------------+------------+------------------+-----------+---------------+--------------+-----------+------------+----------+--------------+--------------------+--------------------+----+---------------+-----------------+
|away_team_id| away_team_name|away_team_score|home_team_id|  home_team_name|home_team_score|            match_id|match_period| match_competition|match_round|   match_season|match_seasonId|match_skyId|match_status|match_type|match_venue_id|match_venue_location|    match_venue_name|_c18|      team_name|total_match_score|
+------------+---------------+---------------+------------+----------------+---------------+--------------------+------------+------------------+-----------+---------------+--------------+-----------+------------+----------+--------------+--------------------+--------------------+----+---------------+-----------------+
|         t25|  Middlesbrough|       

Create a view

In [149]:
match_data.createOrReplaceTempView("match_data")

Total number of teams in the league

In [148]:
spark.sql("SELECT COUNT(DISTINCT(team_name)) AS total_number_of_teams FROM match_data").show()

+---------------------+
|total_number_of_teams|
+---------------------+
|                   24|
+---------------------+



Team with the highest average number of goals per match

In [173]:
spark.sql("SELECT home_team_id, home_team_name, \
            AVG(home_team_score) as average_home_team_score, away_team_id, away_team_name, \
            AVG(away_team_score) as average_away_team_score FROM match_data \
            GROUP BY home_team_id, home_team_name, away_team_id, away_team_name \
            ORDER BY average_home_team_score DESC").show()

+------------+------------------+-----------------------+------------+------------------+-----------------------+
|home_team_id|    home_team_name|average_home_team_score|away_team_id|    away_team_name|average_away_team_score|
+------------+------------------+-----------------------+------------+------------------+-----------------------+
|         t45|       NorwichCity|                    7.0|         t38|  HuddersfieldTown|                    0.0|
|         t94|         Brentford|                    7.0|        t112|  WycombeWanderers|                    2.0|
|         t57|           Watford|                    6.0|        t113|       BristolCity|                    0.0|
|          t9|      CoventryCity|                    6.0|        t103|          Millwall|                    1.0|
|         t19|SheffieldWednesday|                    5.0|         t97|       CardiffCity|                    0.0|
|          t5|   BlackburnRovers|                    5.0|        t112|  WycombeWanderers

Most common match score

In [215]:
spark.sql("SELECT away_team_score, \
            COUNT(away_team_score) AS away_team_value_occurrence, \
            home_team_score, \
            COUNT(away_team_score) AS home_team_value_occurrence \
            FROM match_data \
            GROUP BY away_team_score, home_team_score \
            ORDER BY away_team_value_occurrence DESC, home_team_value_occurrence DESC \
            LIMIT 1").show()

+---------------+--------------------------+---------------+--------------------------+
|away_team_score|away_team_value_occurrence|home_team_score|home_team_value_occurrence|
+---------------+--------------------------+---------------+--------------------------+
|              1|                        72|              1|                        72|
+---------------+--------------------------+---------------+--------------------------+



Avenue with the highest number of goals

In [216]:
spark.sql("SELECT match_venue_id, match_venue_name, \
            COUNT(total_match_score) as highest_venue_score FROM match_data \
            GROUP BY match_venue_id, match_venue_name ORDER BY highest_venue_score DESC").show()

+--------------+--------------------+-------------------+
|match_venue_id|    match_venue_name|highest_venue_score|
+--------------+--------------------+-------------------+
|            57|St.Andrew'sTrilli...|                 46|
|          2867|      LibertyStadium|                 24|
|          2734|     VitalityStadium|                 24|
|         14923|BrentfordCommunit...|                 24|
|            53|             Oakwell|                 24|
|          5040|  CardiffCityStadium|                 23|
|             4|          AshtonGate|                 23|
|            40|       TheCityGround|                 23|
|            32|           EwoodPark|                 23|
|           101|              TheDen|                 23|
|            61|          CarrowRoad|                 23|
|           100|      KenilworthRoad|                 23|
|            41|        Hillsborough|                 23|
|            54|  JohnSmith'sStadium|                 23|
|            4

In [ ]:
# spark.sql("SELECT away_team_id, home_team_id, sum(cast((match_data.away_team_score + match_data.home_team_score) as BIGINT)) \
#             from match_data where away_team_id == home_team_id group by away_team_id, home_team_id").show()

Number of matches played both home and away

In [261]:
spark.sql("SELECT team_name, COUNT(away_team_id) AS away_matches_played, \
          COUNT(home_team_id) AS home_matches_played \
          FROM match_data \
          GROUP BY team_name").show()

+------------------+-------------------+-------------------+
|         team_name|away_matches_played|home_matches_played|
+------------------+-------------------+-------------------+
|       NorwichCity|                 23|                 23|
|SheffieldWednesday|                 23|                 23|
|         LutonTown|                 23|                 23|
|          Millwall|                 23|                 23|
|         Brentford|                 24|                 24|
| QueensParkRangers|                 23|                 23|
|  HuddersfieldTown|                 23|                 23|
|          Barnsley|                 24|                 24|
|         StokeCity|                 23|                 23|
|       Bournemouth|                 24|                 24|
|       BristolCity|                 23|                 23|
|   RotherhamUnited|                 23|                 23|
|       DerbyCounty|                 23|                 23|
|  NottinghamForest|    

Points from a match

In [289]:
spark.sql("SELECT team_name, away_team_score, \
          home_team_score, \
          CASE \
            WHEN away_team_score > home_team_score THEN '3 points away' \
            WHEN away_team_score == home_team_score THEN 'Draw' \
            ELSE '3 points home' \
          END AS Points \
          FROM match_data \
          GROUP BY team_name, away_team_score, home_team_score").show()

+-----------------+---------------+---------------+-------------+
|        team_name|away_team_score|home_team_score|       Points|
+-----------------+---------------+---------------+-------------+
|      Bournemouth|              3|              1|3 points away|
|         Millwall|              2|              1|3 points away|
|  RotherhamUnited|              1|              2|3 points home|
|   BirminghamCity|              0|              3|3 points home|
|   BirminghamCity|              2|              5|3 points home|
|      CardiffCity|              2|              1|3 points away|
|        StokeCity|              1|              4|3 points home|
|      BristolCity|              2|              0|3 points away|
|  PrestonNorthEnd|              1|              0|3 points away|
| NottinghamForest|              2|              1|3 points away|
|  RotherhamUnited|              2|              3|3 points home|
|      CardiffCity|              0|              2|3 points home|
|      Nor

Total Number of Goals Scored by Teams

In [283]:
spark.sql("SELECT away_team_id, team_name, sum(away_team_score) as total_goal_scored\
          FROM match_data as a \
          where a.away_team_name = a.team_name \
          GROUP BY away_team_id, team_name \
          ORDER BY total_goal_scored DESC").show()

+------------+-----------------+-----------------+
|away_team_id|        team_name|total_goal_scored|
+------------+-----------------+-----------------+
|         t94|        Brentford|               40|
|         t45|      NorwichCity|               36|
|         t91|      Bournemouth|               34|
|         t80|      SwanseaCity|               30|
|         t97|      CardiffCity|               29|
|         t37|         Barnsley|               29|
|        t107|  PrestonNorthEnd|               28|
|        t113|      BristolCity|               28|
|          t5|  BlackburnRovers|               28|
|         t52|QueensParkRangers|               25|
|        t108|          Reading|               25|
|         t25|    Middlesbrough|               25|
|        t103|         Millwall|               23|
|        t112| WycombeWanderers|               22|
|         t38| HuddersfieldTown|               22|
|        t110|        StokeCity|               21|
|         t57|          Watford

Save to file

In [288]:
match_data.write.csv("/content/drive/MyDrive/sky_bet_competition/processed_data.csv", header=True)

Why I decided to use Pyspark libraries:

1.   I prefer using a distributed approach whenever I am doing data analysis.
2.   Last but not the least, the compute and storage capabilities are more advantageous.



